https://scikit-learn.org/1.1/modules/decomposition.html#principal-component-analysis-pca

## Training and Logging

In [1]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
(X.shape, y.shape)

((150, 4), (150,))

In [2]:
[(X[0], y[0]), (X[1], y[1])]

[(array([5.1, 3.5, 1.4, 0.2]), 0), (array([4.9, 3. , 1.4, 0.2]), 0)]

In [3]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)
X_r[0]

array([-2.68412563,  0.31939725])

In [4]:
import getpass

import mlflow
from liga.sklearn.mlflow import log_model
from sklearn.decomposition import PCA


mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# train a model
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model = PCA(n_components=2)
    model.fit(X, y)
    registered_model_name = f"{getpass.getuser()}_pca"
    log_model(model, registered_model_name=registered_model_name)


/home/da/.cache/pants/named_caches/pex_root/venvs/d6a7829465a74d6f7ee1c45b5eb7d6677ce27680/cc443c29ee413022004e366aaeb20446296388d4/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/01/06 11:12:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'da_pca'.
2023/01/06 11:12:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: da_pca, version 1
Created version '1' of model 'da_pca'.
/tmp/pants-sandbox-iNPSq8/python/rikai/spark/sql/codegen/mlflow_logger.py:144: UserWarning: value of rikai.output.schema is None or empty and will not be populated to MLflow
  warnings.warn(


## Apply the model on large scale dataset

In [5]:
from example import spark

from rikai.spark.sql.codegen.mlflow_logger import CONF_MLFLOW_TRACKING_URI
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m USING 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)


23/01/06 11:12:57 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.31.194 instead (on interface wlx1cbfce3ffbfe)
23/01/06 11:12:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/da/.cache/pants/named_caches/pex_root/installed_wheels/8f254bc20b539246427b2913639b8a0258db76ab54ba91fbbebb8dc8c36183c1/pyspark-3.3.1-py2.py3-none-any.whl/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/da/.ivy2/cache
The jars for the packages stored in: /home/da/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-293f1ec3-8728-4f9d-83f3-aeba70fa1104;1.0
	confs: [default]
	found ai.eto#rikai_2.12;0.2.0-SNAPSHOT in local-ivy-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found com.typesafe.scala-logging#scala-logging_2.12;3.9.4 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found org.mlflow#mlflow-client;1.21.0 in central
	found org.apache.logging.log4j#log4j-core;2.17.1 in central
:: resolution report :: resolve 238ms :: artifacts dl 13ms
	:: modules in use:
	ai.eto#rikai_2.12;0.2.0-SNAPSHOT from local-ivy-cache in [default]
	com.typesafe.scala-logging#scala-logging_2.12;3.9.4 from central in [default]
	org.apache.logging.log4j#log4j-core;2.17.1 from central in [default]
	org.mlflow#mlflow-client;1.21.0 from central in [default]
	org.slf4j#slf4j-api;1.7.30 from

23/01/06 11:13:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4048. Attempting port 4049.
23/01/06 11:13:04 WARN SparkContext: The path file:///home/da/.ivy2/jars/ai.eto_rikai_2.12-0.2.0-SNAPSHOT.jar

In [6]:
from liga.numpy.sql import literal

result = spark.sql(f"""
select
  ML_PREDICT(mlflow_sklearn_m, {literal(X[0])})
"""
)

result.printSchema()
result.toPandas()

root
 |-- mlflow_sklearn_m: array (nullable = true)
 |    |-- element: float (containsNull = true)



,mlflow_sklearn_m
0,"[-2.6841256618499756, 0.3193972408771515]"
